In [11]:
#importa o pandas e como vou chma
import pandas as pd
# Defina o caminho para o arquivo CSV bruto
file_path = '../data/raw/rawAgroMilho.csv'

NUMERO_DE_LINHAS_DO_CABECALHO = 4
NUMERO_DE_LINHAS_DO_RODAPE = 37

# Usa o pandas para ler o arquivo csv e o carrega em uma tabela (DataFrame)
df = pd.read_csv(file_path,
                 skiprows=NUMERO_DE_LINHAS_DO_CABECALHO,
                 skip_blank_lines=True,
                 na_values=['-'],
                 engine='python', 
                 skipfooter=NUMERO_DE_LINHAS_DO_RODAPE)

# Mostra o topo do DataFrame (os primeiros 5 registros)

df.head()

,Cód.,Município,Milho (em grão),Milho (em grão).1,Milho (em grão).2,Milho (em grão).3,Milho (em grão).4,Milho (em grão).5,Milho (em grão).6
0,5000203,Água Clara (MS),NaN,NaN,NaN,NaN,342,120,NaN
1,5000252,Alcinópolis (MS),4620,4680,6824,7400,7400,7400,7500
2,5000609,Amambai (MS),45000,45000,46000,46000,65000,70000,70000
3,5000708,Anastácio (MS),4550,4950,4950,4107,7230,8617,9885
4,5000807,Anaurilândia (MS),8050,12250,9080,7500,8500,6500,8000


In [ ]:
import pandas as pd
import io

file_path = '../data/raw/rawAgroMilho.csv'

# Com base na legenda, definimos todos os símbolos que significam "não há dados"
VALORES_FALTANTES = ['-', '...', 'X', '..']

# ETAPA 1: Ler o arquivo inteiro para a memória, linha por linha
with open(file_path, 'r', encoding='utf-8') as f:
    todas_as_linhas = f.readlines()

# ETAPA 2: Função ajustada para ser mais precisa na busca pelo cabeçalho e dados
def extrair_tabela_por_variavel(linhas_do_arquivo: list, nome_da_variavel: str) -> pd.DataFrame:
    """
    Esta função encontra a tabela de uma variável específica dentro do arquivo de relatório,
    a extrai e a retorna como um DataFrame limpo.
    """
    bloco_de_dados = []
    cabecalho = None
    
    # Encontra o índice da linha que define a nossa variável de interesse
    try:
        indice_inicio_bloco = [i for i, linha in enumerate(linhas_do_arquivo) if nome_da_variavel in linha][0]
    except IndexError:
        raise ValueError(f"Não foi possível encontrar o bloco da variável: {nome_da_variavel}")

    # A partir da definição da variável, procuramos pela linha de cabeçalho real
    for i in range(indice_inicio_bloco, len(linhas_do_arquivo)):
        linha_atual = linhas_do_arquivo[i]
        # O cabeçalho real é o que começa com "Cód.","Município" e contém um ano como "2017"
        if linha_atual.strip().startswith('"Cód.","Município"') and '"2017"' in linha_atual:
            cabecalho = linha_atual
            # Os dados reais começam 2 linhas após o cabeçalho real (pulando a linha do "Milho (em grão)")
            inicio_dos_dados = i + 2
            break
    
    if not cabecalho:
        raise ValueError(f"Não foi possível encontrar a linha de cabeçalho para a variável: {nome_da_variavel}")

    # Coletamos as linhas de dados até encontrar o fim do bloco
    for linha_de_dados in linhas_do_arquivo[inicio_dos_dados:]:
        if "Fonte: IBGE" in linha_de_dados:
            break # Para o loop quando o bloco termina
        bloco_de_dados.append(linha_de_dados)

    # Juntamos o cabeçalho e os dados em uma única string de CSV
    csv_limpo_string = cabecalho + "".join(bloco_de_dados)

    # Usamos o Pandas para ler esta string, que agora é um CSV limpo
    df = pd.read_csv(io.StringIO(csv_limpo_string), na_values=VALORES_FALTANTES)
    
    return df

# ETAPA 3: Usar nossa função para extrair e limpar cada tabela que queremos
df_area_bruta = extrair_tabela_por_variavel(todas_as_linhas, "Área plantada ou destinada à colheita")
df_prod_bruta = extrair_tabela_por_variavel(todas_as_linhas, "Quantidade produzida")
df_rend_bruta = extrair_tabela_por_variavel(todas_as_linhas, "Rendimento médio da produção")

# ETAPA 4: Função de transformação (nenhuma mudança aqui)
def transformar_dataframe(df_bruto, nome_da_coluna_valor: str) -> pd.DataFrame:
    df_bruto.rename(columns={'Cód.': 'cod_municipio', 'Município': 'municipio_uf'}, inplace=True)
    df_long = pd.melt(df_bruto, 
                      id_vars=['cod_municipio', 'municipio_uf'], 
                      var_name='ano', 
                      value_name=nome_da_coluna_valor)
    
    df_clean = df_long.dropna(subset=[nome_da_coluna_valor])
    df_clean['ano'] = pd.to_numeric(df_clean['ano'])
    df_clean[nome_da_coluna_valor] = pd.to_numeric(df_clean[nome_da_coluna_valor])
    
    df_clean[['municipio_nome', 'uf']] = df_clean['municipio_uf'].str.extract(r'^(.*) \((.*)\)$')
    colunas_finais = ['cod_municipio', 'municipio_nome', 'uf', 'ano', nome_da_coluna_valor]
    df_final = df_clean[colunas_finais].copy()
    
    return df_final

# ETAPA 5: Executar a transformação e unificação (nenhuma mudança aqui)
df_area = transformar_dataframe(df_area_bruta, 'area_plantada_ha')
df_prod = transformar_dataframe(df_prod_bruta, 'quantidade_produzida_ton')
df_rend = transformar_dataframe(df_rend_bruta, 'rendimento_medio_kg_ha')

df_merged = pd.merge(df_area, df_prod, on=['cod_municipio', 'ano', 'municipio_nome', 'uf'], how='inner')
df_final = pd.merge(df_merged, df_rend, on=['cod_municipio', 'ano', 'municipio_nome', 'uf'], how='inner')

print("TODOS OS DADOS FORAM PROCESSADOS E UNIFICADOS COM SUCESSO!")
df_final.info()
df_final.head()

TODOS OS DADOS FORAM PROCESSADOS E UNIFICADOS COM SUCESSO!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3025 entries, 0 to 3024
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cod_municipio             3025 non-null   int64  
 1   municipio_nome            3025 non-null   object 
 2   uf                        3025 non-null   object 
 3   ano                       3025 non-null   int64  
 4   area_plantada_ha          3025 non-null   float64
 5   quantidade_produzida_ton  3025 non-null   float64
 6   rendimento_medio_kg_ha    3025 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 165.6+ KB


/tmp/ipykernel_1461645/3875566828.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['ano'] = pd.to_numeric(df_clean['ano'])
/tmp/ipykernel_1461645/3875566828.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[nome_da_coluna_valor] = pd.to_numeric(df_clean[nome_da_coluna_valor])
/tmp/ipykernel_1461645/3875566828.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

,cod_municipio,municipio_nome,uf,ano,area_plantada_ha,quantidade_produzida_ton,rendimento_medio_kg_ha
0,5000252,Alcinópolis,MS,2017,4620.0,30780.0,6662.0
1,5000609,Amambai,MS,2017,45000.0,248400.0,5520.0
2,5000708,Anastácio,MS,2017,4550.0,16320.0,3587.0
3,5000807,Anaurilândia,MS,2017,8050.0,43450.0,5398.0
4,5000856,Angélica,MS,2017,2800.0,14000.0,5000.0


In [5]:
# Definir o caminho para salvar nosso arquivo limpo
caminho_processado = '../data/processed/dados_consolidados_produtividade_co_2017-2023.csv'

# Salvar o DataFrame em um novo CSV, sem o índice do pandas
df_final.to_csv(caminho_processado, index=False)

print(f"DataFrame limpo salvo com sucesso em: {caminho_processado}")

DataFrame limpo salvo com sucesso em: ../data/processed/dados_consolidados_produtividade_co_2017-2023.csv
